In [1]:
""" Scrape Google search with Selenium """

!killall -q chromedriver
!killall -q chrome

import time
time.sleep(2)

from PIL import Image

from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver import ActionChains

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup

options = Options()
options.add_argument("--headless") # Runs Chrome in headless mode.
options.add_argument('window-size=1080x2700') # setting window size is optional
options.add_argument('--no-sandbox') # # Bypass OS security model
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")
options.add_argument("--disable-setuid-sandbox")

service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(options=options, service=service)

def is_prompt(driver):
    try:
        if driver.find_element(By.XPATH, "/html/body/div[3]/div[3]/span/div/div/div[3]/button[2]/div[contains(text(), 'Hyväksyn')]"):
            driver.save_screenshot("prompt.png")
            return True
    except NoSuchElementException:
        return False

def wait_page_load(driver):
    delay = 4 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.g")))
        time.sleep(2)
        if is_prompt(driver):
            continue_elem = driver.find_element(By.XPATH, "/html/body/div[3]/div[3]/span/div/div/div[3]/button[2]/div[contains(text(), 'Hyväksyn')]")
            continue_elem.click()
            time.sleep(2)
    except TimeoutException:
        print("Loading took too much time!")
        driver.quit()
        exit()

def take_screenshot(driver):
    driver.save_screenshot("google.png")
    image = Image.open("google.png")
    image.show()

def to_html(webelem):
    soup = BeautifulSoup(webelem.get_attribute('outerHTML'), "html.parser")
    return soup

def get_google_page_snippets(driver, page_number):
    url = "https://www.google.com/search?q=carbon+footprint&start={0}".format(page_number*10)
    driver.get(url)
    wait_page_load(driver)
    #take_screenshot(driver)
    all_snippets = driver.find_elements(By.XPATH, "//div[@class='g']/div/div/div/div[1]/span[not(@class)]")
    all_snippets_html = [to_html(ele) for ele in all_snippets]
    all_snippets_text = [ snippet.text for snippet in all_snippets_html ]
    return all_snippets_text

def flatten(t):
    return [item for sublist in t for item in sublist]

paged_all_snippets = [ get_google_page_snippets(driver, idx) for idx in range(1, 12) ]

all_snippets = flatten(paged_all_snippets)
print(*all_snippets[:15], sep='\n')
print(len(all_snippets))

driver.quit()

""" 
Doesn't work reliably on deepnote
Locally runs just fine

Omits first hit on first page. This is due to the XPATH selector, fix it if you can
"""


Use the World's most popular online carbon footprint calculator, and it's FREE. Calculate your carbon emissions from Buildings, Cars, Flights and other ...
According to WHO, a carbon footprint is a measure of the impact your activities have on the amount of carbon dioxide (CO2) produced through the ...
Carbon footprint, amount of carbon dioxide emissions associated with all the activities of a person or other entity. It includes direct emissions, ...
Amazon's corporate carbon footprint quantifies the total greenhouse gas emissions attributed to our direct and indirect operational activities.
Reduce carbon footprint with these handy tips. Calculate your emissions & buy offsets that change lives in regions where incomes are less than $2/day.
A carbon footprint measures the total greenhouse gas emissions caused directly and indirectly by a person, organisation, event or product.
Oil companies have made a carbon footprint your problem, but what about their own massive carbon footprints?
Th

" \nDoesn't work reliably on deepnote\nLocally runs just fine\n\nOmits first hit on first page. This is due to the XPATH selector, fix it if you can\n"

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=df8df42c-8503-4bb3-8c9d-497cf36bf82c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>